<a href="https://colab.research.google.com/github/gabbypaola/DATA608/blob/main/Module4/Module4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [4]:
! pip install dash   
! pip install dash-html-components                                         
! pip install dash-core-components                                     
! pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 4.3 MB/s 
     |████████████████████████████████| 357 kB 38.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import plotly.graph_objects as go
import plotly.express as px
import plotly as py

In [6]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until


This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [7]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


Looks good, but lets take a look at the shape of this data:

In [8]:
trees.shape

(1000, 45)

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset. 

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [9]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29,739,4052307,4022210001
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19,973,4101931,4044750045
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34,449,3338310,3028870001
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34,449,3338342,3029250001
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39,165,3025654,3010850052


If we wanted the next 5, we would use this url:

In [10]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,988418.6997,219825.5227,3.0,145.0,1076229.0,1.011310e+09
1,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,988311.1900,219885.2785,3.0,145.0,1076229.0,1.011310e+09
2,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,987769.1163,217157.8561,3.0,133.0,1086093.0,1.010410e+09
3,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,963073.1998,156635.5542,NaN,NaN,NaN,NaN
4,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,992653.7253,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


You can read more about paging using the Socrata API [here](https://dev.socrata.com/docs/paging.html)

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [11]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,spc_common
0,4619,NaN
1,43,black walnut
2,9,spruce
3,62,tulip-poplar
4,7,trident maple
...,...,...
128,195,crimson king maple
129,774,purple-leaf plum
130,363,sawtooth oak
131,505,willow oak


This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

----------

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [12]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

---

**Instructions:**  
Build a dash app for an arborist studying the health of various tree species (as defined by the variable `spc_common`) across each borough (defined but he variable `borough`). This arborist would like to answer the following two questions:  
1. What proportion of trees are in good, fair, or poor health according to the health vairable?  
2. Are stewards (steward activity measured by the `steward` variable) having an impact on the health of trees?
  
**Deployment:** Dash deployment is more complicated than deploying shiny apps, so deployment in this case is **optional** (and will result in extra credit). You can read instructoins on deploying a dash app to heroku here: https://pash.ploy.ly/deployment

**1.**

In [13]:
soql_1 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,boroname,health,count(spc_common)' +\
        '&$group=spc_common,boroname,health')
df = pd.read_json(soql_1 + '&$limit=700000')
df_1 = df.dropna() #drop NA values
df_1 = df_1.rename(columns={"spc_common": "species", "boroname": "borough", "count_spc_common": "count"})

In [14]:
df_1

,species,borough,health,count
0,American beech,Bronx,Fair,7
1,American beech,Bronx,Good,21
2,American beech,Bronx,Poor,3
3,American beech,Brooklyn,Fair,18
4,American beech,Brooklyn,Good,59
...,...,...,...,...
1805,willow oak,Queens,Good,650
1806,willow oak,Queens,Poor,15
1807,willow oak,Staten Island,Fair,37
1808,willow oak,Staten Island,Good,210


In [15]:
pd.crosstab(index=df_1['health'], columns='count', margins=True) 

col_0,count,All
health,,
Fair,616,616
Good,649,649
Poor,543,543
All,1808,1808


**2.**

In [16]:
soql_2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,boroname,health,steward,count(spc_common)' +\
        '&$group=spc_common,boroname,health,steward')
df2 = pd.read_json(soql_2 + '&$limit=700000')
df_2 = df2.dropna()
df_2 = df_2.rename(columns={"spc_common": "species", "boroname": "borough", "count_spc_common":"count"})

In [17]:

df_2

,species,borough,health,steward,count
0,Atlantic white cedar,Bronx,Good,None,36
1,purple-leaf plum,Staten Island,Fair,1or2,87
2,catalpa,Bronx,Good,4orMore,1
3,kousa dogwood,Queens,Good,None,55
4,Norway spruce,Staten Island,Good,1or2,6
...,...,...,...,...,...
4560,ash,Staten Island,Good,3or4,1
4561,sweetgum,Staten Island,Poor,None,125
4562,American linden,Manhattan,Good,4orMore,20
4563,white pine,Brooklyn,Fair,None,8


In [18]:
df_2.species.nunique()

132

In [19]:
# t2.steward.nunique()
df_2.steward.unique().tolist()

['None', '1or2', '4orMore', '3or4']

In [20]:
#frequecy counts of each value
df_2.steward.value_counts()

None       1719
1or2       1586
3or4        917
4orMore     332
Name: steward, dtype: int64

In [21]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4554 entries, 0 to 4564
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   species  4554 non-null   object
 1   borough  4554 non-null   object
 2   health   4554 non-null   object
 3   steward  4554 non-null   object
 4   count    4554 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 213.5+ KB


In [22]:
def new_col(row):
  if row['steward'] == 'None':
    return 'No'
  else :
    return 'Yes'

df_2['steward_present'] = df_2.apply(new_col, axis=1)

In [23]:
df_3 = df_2.copy().drop(['steward'], axis=1)

In [24]:
df_3
#df_3.groupby(['species','borough','health','steward_present', 'count']).size().reset_index(name='count')

,species,borough,health,count,steward_present
0,Atlantic white cedar,Bronx,Good,36,No
1,purple-leaf plum,Staten Island,Fair,87,Yes
2,catalpa,Bronx,Good,1,Yes
3,kousa dogwood,Queens,Good,55,No
4,Norway spruce,Staten Island,Good,6,Yes
...,...,...,...,...,...
4560,ash,Staten Island,Good,1,Yes
4561,sweetgum,Staten Island,Poor,125,No
4562,American linden,Manhattan,Good,20,Yes
4563,white pine,Brooklyn,Fair,8,No


In [25]:
df_4 = df_3.groupby(['species','borough','health','steward_present'])['count'].sum().to_frame().reset_index()

In [26]:
df_4

,species,borough,health,steward_present,count
0,'Schubert' chokecherry,Bronx,Fair,No,67
1,'Schubert' chokecherry,Bronx,Fair,Yes,23
2,'Schubert' chokecherry,Bronx,Good,No,301
3,'Schubert' chokecherry,Bronx,Good,Yes,155
4,'Schubert' chokecherry,Bronx,Poor,No,25
...,...,...,...,...,...
3335,willow oak,Staten Island,Fair,Yes,11
3336,willow oak,Staten Island,Good,No,177
3337,willow oak,Staten Island,Good,Yes,33
3338,willow oak,Staten Island,Poor,No,9


In [27]:
#get frequency of each value (0=no, 1=yes)
df_2.steward_present.value_counts()

Yes    2835
No     1719
Name: steward_present, dtype: int64

In [28]:
pd.crosstab(index=df_2['health'], columns=df_2['steward'], margins=True)

steward,1or2,3or4,4orMore,None,All
health,,,,,
Fair,524,291,79,589,1483
Good,622,464,220,637,1943
Poor,440,162,33,493,1128
All,1586,917,332,1719,4554


In [29]:
pd.crosstab(index=df_2['health'], columns=df_2['steward_present'], margins=True)

steward_present,No,Yes,All
health,,,
Fair,589,894,1483
Good,637,1306,1943
Poor,493,635,1128
All,1719,2835,4554


In [30]:
#1 viz
species = 'American beech' #only dynamic filter needed in the dash would be tree type
df_plot1 = df_1[(df_1['species'] == species)] 

fig = px.histogram(df_plot1, x='borough', y='count', color='health', #colors depend on health rating
                   category_orders={"health": ["Good","Fair", "Poor"]}, #organize health rating values
                   color_discrete_sequence=["#109618", "#FF9900", "#DC3912"], #assign and orgnize colors to match ratings
title='American Beech tree Health throughout the 5 Boroughs',
labels={"borough": "Borough","count": "Count"})

fig.update_xaxes(type='category', title_text='Borough', showline=True, linewidth=.5, linecolor='lightgrey')
fig.update_yaxes(title_text='Tree Count', showline=True, linewidth=.2, linecolor='lightgrey', gridwidth=.2 ,gridcolor='lightgrey')
fig.update_layout(barmode='group', #plots all boroughs in a grouped bar chart for the selected tree type
                  xaxis={'categoryorder':'total descending'}, 
                  legend_title='Health Status', 
                  plot_bgcolor= 'rgba(0, 0, 0, 0)')

fig.show()

In [31]:
#2 viz
species = 'American beech' #only dynamic filter needed in the dash would be tree type
df_plot2 = df_4[(df_4['species'] == species)] 

In [32]:
rating_color = {"#FF9900":"Fair","#109618":"Good","#DC3912": "Poor"}

fig = go.Figure()

fig.update_layout(barmode='stack', width=900, height=800, uniformtext_minsize=8, 
                  uniformtext_mode='hide', plot_bgcolor= 'rgba(0, 0, 0, 0)', 
                  legend_title='Health Status')
fig.update_xaxes(linewidth=.5, linecolor='lightgrey')
fig.update_yaxes(linewidth=.2, linecolor='lightgrey', gridwidth=.2 ,gridcolor='lightgrey')

for r, c in zip(df_plot2['health'].unique(), rating_color):
    df_plot = df_plot2[df_plot2['health'] == r]
    fig.add_trace(
        go.Bar(
            x=[df_plot['borough'], df_plot['steward_present']],
            y=df_plot['count'],
            marker_color=c,
            name=r,
            text=df_plot['count'],
            width=0.45,
            textposition='auto')
    )

fig.show()

In [ ]:
app = dash.Dash(__name__, external_stylesheets='yeti')

species = df_1['species'].unique()

app.layout = html.Div(children=[
    html.H1(children = 'NYC Tree Health and Stewardship Effectiveness'),
    html.P(children = 'Tree health and stewardship impact plots are generated after selecting a tree species. Data obtained from the NYC open data using SOQL commands.'),    
    html.P(children = 'Select a tree species:'),
    dcc.Dropdown(
        id='species',
        options=[{'label': s, 'value': s} for s in species],
        value=species[0] #default
    ),
    #Q1
    html.Div(id='health-app'),
    html.H2(children = 'Tree Health comparison'),
    dcc.Graph(id='health-bar'),
    #Q2
    html.Div(id='steward-app'),
    html.H2(children = 'Impact of Stewardship on Tree Health'),
    dcc.Graph(id='steward'),
    html.P(children = 'This visualization can be used to compare stewardship effectivness on tree health across the 5 boroughs'),

])

##Q1##

@app.callback(
    dash.dependencies.Output('health-bar', 'figure'), 
    [dash.dependencies.Input('species', 'value')]
)

def health_app(species):
    #filter 
    df_plot1 = df_1[(df_1['species'] == species)] 
    
    #plot begin
    fig = px.histogram(df_plot1, x='borough', y='count', color='health', #colors depend on health rating
                   category_orders={"health": ["Good","Fair", "Poor"]}, #organize health rating values
                   color_discrete_sequence=["#109618", "#FF9900", "#DC3912"], #assign and orgnize colors to match ratings
    title='American Beech tree Health throughout the 5 Boroughs',
    labels={"borough": "Borough","count": "Count"})

    fig.update_xaxes(type='category', title_text='Borough', showline=True, linewidth=.5, linecolor='lightgrey')
    fig.update_yaxes(title_text='Tree Count', showline=True, linewidth=.2, linecolor='lightgrey', gridwidth=.2 ,gridcolor='lightgrey')
    fig.update_layout(barmode='group', #plots all boroughs in a grouped bar chart for the selected tree type
                  xaxis={'categoryorder':'total descending'}, 
                  legend_title='Health Status', 
                  plot_bgcolor= 'rgba(0, 0, 0, 0)')
    return fig
    #plot end

##Q2##

@app.callback(
    dash.dependencies.Output('steward', 'figure'), 
    [dash.dependencies.Input('species', 'value')]
)

def species_app(species):
    #filter
    df_plot2 = df_4[(df_4['species'] == species)]
    
    #begin plot

    rating_color = {"#FF9900":"Fair","#109618":"Good","#DC3912": "Poor"}

    fig2 = go.Figure()

    fig2.update_layout(barmode='stack', width=900, height=800, uniformtext_minsize=8, 
                  uniformtext_mode='hide', plot_bgcolor= 'rgba(0, 0, 0, 0)', 
                  legend_title='Health Status')
    fig2.update_xaxes(linewidth=.5, linecolor='lightgrey')
    fig2.update_yaxes(linewidth=.2, linecolor='lightgrey', gridwidth=.2 ,gridcolor='lightgrey')

    for r, c in zip(df_plot2['health'].unique(), rating_color):
        df_plot = df_plot2[df_plot2['health'] == r]
        fig.add_trace(
            go.Bar(
                x=[df_plot['borough'], df_plot['steward_present']],
                y=df_plot['count'],
                marker_color=c,
                name=r,
                text=df_plot['count'],
                width=0.45,
                textposition='auto')
        )

        
    return fig2

    #end plot

if __name__ == '__main__':
       app.run_server(debug=False)



**References**

https://austinlasseter.medium.com/how-to-deploy-a-simple-plotly-dash-app-to-heroku-622a2216eb73
https://towardsdatascience.com/deploying-your-dash-app-to-heroku-the-magical-guide-39bd6a0c586c
https://towardsdatascience.com/creating-an-interactive-dashboard-from-jupyter-notebook-with-voila-b64918b4d15a

https://medium.com/@moritzkoerber/how-to-plot-a-grouped-stacked-bar-chart-in-plotly-df1685b83460
https://plotly.com/python/discrete-color/
https://community.plotly.com/t/grouped-stacked-bar-chart/60805

viz #2: https://stackoverflow.com/questions/73124895/stacked-and-grouped-barchart-with-plotly

